In [2]:
from PyPDF2 import PdfReader

def load_pdf_chunks(file_path, chunk_size=500):
    reader = PdfReader(file_path)
    full_text = ""
    for page in reader.pages:
        full_text += page.extract_text() or ""
    chunks = [full_text[i:i + chunk_size] for i in range(0, len(full_text), chunk_size)]
    return chunks

pdf_chunks = load_pdf_chunks("womens_health_book.pdf")

In [3]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")

genai.configure(api_key=api_key)

def get_embedding(text):
    model = "models/text-embedding-004"
    response = genai.embed_content(
        model=model,
        content=text,
        task_type="retrieval_document"
    )
    return response['embedding']

embedding_data = []
for chunk in pdf_chunks:
    vector = get_embedding(chunk)
    embedding_data.append({"text": chunk, "embedding": vector})

In [4]:
import chromadb
from chromadb.config import Settings

client = chromadb.Client(Settings(anonymized_telemetry=False))
collection = client.create_collection(name="womens_health")

for idx, item in enumerate(embedding_data):
    collection.add(
        documents=[item["text"]],
        embeddings=[item["embedding"]],
        ids=[str(idx)]
    )

In [5]:

def handle_small_talk(prompt):
    small_talk_responses = {
        "hi": "Hello! How can I help you today?",
        "hello": "Hi there! How can I assist you?",
        "how are you": "I'm just a chatbot, but I'm here and ready to help!",
        "good morning": "Good morning! How can I assist you today?",
        "good evening": "Good evening! Feel free to ask me anything.",
        "thanks": "You're welcome!",
        "thank you": "You're welcome!",
        "bye": "Goodbye! Take care.",
    }

    prompt_clean = prompt.lower().strip()
    return small_talk_responses.get(prompt_clean)


In [6]:
def ask_question(question, top_k=3):
    small_talk = handle_small_talk(question)
    if small_talk:
        return small_talk

    results = collection.query(
        query_embeddings=[get_embedding(question)],
        n_results=top_k
    )
    context = "\n\n".join(results["documents"][0])

    prompt = f"""
    You are a helpful and friendly women's health assistant.
    Use the following information to answer health-related questions.
    If the user's message is a greeting or casual, respond appropriately.

    Context:
    {context}

    User: {question}
    """
    model = genai.GenerativeModel("models/gemini-1.5-pro-latest")
    response = model.generate_content(prompt)
    return response.text


In [ ]:
question = "What are the symptoms of PCOS?"
response = ask_question(question)
print(response)

Women with PCOS may experience a variety of symptoms, including irregular or prolonged menstrual periods, excess androgen levels (which can lead to acne, hirsutism, and male-pattern baldness), and the development of numerous cysts on the ovaries.  It's important to note that the intensity of symptoms can range from mild to quite severe, and not all women will experience every symptom.  If you are concerned you might have PCOS, it's crucial to consult a healthcare professional for proper diagnosis and management.



In [10]:
models = genai.list_models()
for m in models:
    print(m.name, m.supported_generation_methods)

models/embedding-gecko-001 ['embedText', 'countTextTokens']
models/gemini-1.0-pro-vision-latest ['generateContent', 'countTokens']
models/gemini-pro-vision ['generateContent', 'countTokens']
models/gemini-1.5-pro-latest ['generateContent', 'countTokens']
models/gemini-1.5-pro-001 ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro-002 ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro ['generateContent', 'countTokens']
models/gemini-1.5-flash-latest ['generateContent', 'countTokens']
models/gemini-1.5-flash-001 ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-flash-001-tuning ['generateContent', 'countTokens', 'createTunedModel']
models/gemini-1.5-flash ['generateContent', 'countTokens']
models/gemini-1.5-flash-002 ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-flash-8b ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-1.5-flash-8b-001 ['createCachedCon